In [1]:
import nltk
nltk.download('punkt')
import joblib
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rissu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import tensorflow as tf
import numpy as np
# import tflearn
import random
import json

In [ ]:
file=open('intents.json')

In [ ]:
intents=json.load(file)

In [ ]:
intents

In [ ]:
words=[]
classes=[]
documents=[]
ignore=['?']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w=nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
wordd=[]
for w in words:
    if w not in ignore:
        wordd.append(stemmer.stem(w.lower()))
        

In [ ]:
wordd=sorted(list(set(wordd)))

In [ ]:
len(documents),len(classes),len(wordd)

In [ ]:
training=[]
output=[]
output_empty=[0]*len(classes)
for doc in documents:
    bag=[]
    pattern_words=doc[0]
    pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
    for w in wordd:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    training.append([bag,output_row])
random.shuffle(training)
training=np.array(training)
train_x=list(training[:,0])
train_y=list(training[:,1])

In [ ]:
# resetting underlying graph data
tf.reset_default_graph()

# building neural network

net=tflearn.input_data(shape=[None,len(train_x[0])])
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net=tflearn.regression(net)

# defining model and setting up tensorboard
model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')

# start training
model.fit(train_x,train_y,n_epoch=1000,batch_size=8,show_metric=True)
model.save('model.tflearn')

In [ ]:
len(train_x[0])

In [ ]:
len(train_x[0])

In [ ]:
len(train_y[0])

In [ ]:
import pickle
pickle.dump({'words':wordd,
             'classes':classes,
             'train_x':train_x,
             'train_y':train_y},open('training_data','wb'))

In [ ]:
data=pickle.load(open('training_data','rb'))

In [ ]:
words=data['words']
classes=data['classes']
train_x=data['train_x']
train_y=data['train_y']

In [ ]:
with open('intents.json') as json_data:
    intents=json.load(json_data)

In [ ]:
model.load('./model.tflearn')

In [ ]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [ ]:
def bag_of_words(sentence,words,show_details=False):
    bag=[0]*len(words)
    sentence=clean_up_sentence(sentence)
    for s in sentence:
        for i,w in enumerate(words):
            if w==s:
                bag[i]=1
                if show_details:
                    print("%s"%w)
    return np.array(bag)

In [ ]:
bag_of_words('What can I do for you?',words,show_details=True)

In [ ]:
ERROR_THRESOLD=0.30
def classify(sentence):
    results=model.predict([bag_of_words(sentence,words)])[0]
    results=[[i,r] for i,r in enumerate(results) if r>ERROR_THRESOLD]
    results.sort(key=lambda x: x[1],reverse=True)
    return_list=[]
    for r in results:
        return_list.append((classes[r[0]],r[1]))
    return return_list
        

In [ ]:
classify('What are your hours of operation')

In [ ]:
def response(sentence,userID='123',show_details=False):
    results=classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag']==results[0][0]:
                    return print(random.choice(i['responses']))
            results.pop(0)
response("Bye Bye")

In [ ]:
response('How can I help you?')

In [ ]:
len(train_x)

In [ ]:
def response(sentence,userID='123',show_details=False):
    results=classify(sentence)

    for i in intents['intents']:
        if i['tag']==results[0][0]:
            return print(random.choice(i['responses']))
    results.pop(0)
response("At what time will restaurant open?")

In [4]:
a=dict()

In [6]:
a[1]=2

In [7]:
a


{1: 2}

In [9]:
a[(1,2,3)]='a'

In [10]:
a

{1: 2, (1, 2, 3): 'a'}